In [1]:
import pandas as pd
from datetime import datetime, timedelta

In [2]:
ruta=r"C:\Users\jcleiva\Documents\Reportes Base"

In [3]:
rDesp=r"C:\Users\jcleiva\Documents\Reportes Base\{}\Despachos\{}. Despachos {}Q.xlsx"

In [4]:
cDesp=["Destinatario de mercancías","Entrega","    # PLU","Material","Marca Formato"]
cvDesp={"Entrega":str,"    # PLU":str,"Material":str}
convEjec={"Centro de beneficio":str,"Número de cuenta":str,"Material":str,"Centro":str}

In [5]:
def ejecCEBEAgg(m,y,ruta):
    colsEjec=["Centro de beneficio","Número de cuenta","Denominación","Material","Centro","Clase mov. MM","En moneda local centro de beneficio","Cantidad"]
    convEjec={"Centro de beneficio":str,"Número de cuenta":str,"Material":str,"Centro":str}
    colsTemp=["Centro de beneficio","Número de cuenta","Denominación","Material","Centro","Clase mov. MM"]
    tiempo=(y,m)
    direc={"Costo de Ventas Industria CEBE.xlsx":"Costo de Ventas",
           "Ingreso Industria CEBE.xlsx":"Ingresos",
           "Gasto Industria CEBE.xlsx":"Gasto",
           "Cuenta 7 Industria CEBE.xlsx":"Ejecución"}
    
    #for file in ["Costo de Ventas Industria CEBE.xlsx","Ingreso Industria CEBE.xlsx","Gasto Industria CEBE.xlsx","Cuenta 7 Industria CEBE.xlsx"]:
    for file in ["Costo de Ventas Industria CEBE.xlsx"]:
        if file=="Costo de Ventas Industria CEBE.xlsx":
            cols=colsEjec+["Documento comercial"]
            conv=convEjec
            conv["Documento comercial"]=str
            cTemp=colsTemp+["Documento comercial"]
        else:
            cols=colsEjec
            conv=convEjec
            cTemp=colsTemp
            
        dfEj=pd.read_excel(ruta+"\{}\{}\{}. {}".format(tiempo[0],direc[file],tiempo[1],file),
                           usecols=cols,
                           converters=conv)
        
        for i in dfEj.columns:
            if dfEj[i].dtype == "object":
                dfEj[i].fillna("",inplace=True)
            elif (dfEj[i].dtype == "float64") or (dfEj[i].dtype == "int64"):
                dfEj[i].fillna(0.0,inplace=True)
        
        dfEj=dfEj.groupby(cTemp,dropna=False).sum().reset_index()
        
        dfEj["Fecha"]=datetime(tiempo[0],tiempo[1],1)
        dfEj=dfEj.rename(columns={"En moneda local centro de beneficio":"Importe"})
        
        dfEj.to_excel(ruta+"\{}\{}\{}. {} (Agg).xlsx".format(tiempo[0],direc[file],tiempo[1],file[:-5]),index=None)
        print("Archivo: "+"\{}\{}\{}. {} (Agg).xlsx".format(tiempo[0],direc[file],tiempo[1],file[:-5])+" generado con éxito")
        
        if file=="Costo de Ventas Industria CEBE.xlsx":
            dfDespQ=pd.read_excel(rDesp.format(y,m,1),converters=cvDesp,usecols=cDesp)
            dfDespQ=pd.concat([dfDespQ,pd.read_excel(rDesp.format(y,m,2),converters=cvDesp,usecols=cDesp)])
            dfDespQ=dfDespQ.drop_duplicates()
            dfDespQ=dfDespQ.rename(columns={"Entrega":"Documento comercial","    # PLU":"PLU Industria"})
        
            if not dfEj.merge(dfDespQ,on=["Documento comercial","Material"],how="left").shape[0]==dfEj.shape[0]:
                raise Exception("Despachos por Q inserta filas")

            dfEj=dfEj.merge(dfDespQ,on=["Documento comercial","Material"],how="left")

            del dfEj["Documento comercial"]
            dfEj["PLU Industria"]=dfEj["PLU Industria"].str.lstrip("0")
            
            for i in dfEj.columns:
                if dfEj[i].dtype == "object":
                    dfEj[i].fillna("",inplace=True)
                elif (dfEj[i].dtype == "float64") or (dfEj[i].dtype == "int64"):
                    dfEj[i].fillna(0.0,inplace=True)
            
            dfEj=dfEj.groupby(['Centro de beneficio', 'Número de cuenta', 'Denominación', 
                               'Material','Centro', 'Clase mov. MM', 'Fecha',
                               'Destinatario de mercancías', 'PLU Industria', 'Marca Formato']).sum().reset_index()
            
            dfEj.to_excel(ruta+"\{}\{}\{}. {} (PLU).xlsx".format(tiempo[0],direc[file],tiempo[1],file[:-5]),index=None)
            print("Archivo: "+"\{}\{}\{}. {} (PLU).xlsx".format(tiempo[0],direc[file],tiempo[1],file[:-5])+" generado con éxito")

In [6]:
dfEj=ejecCEBEAgg(8,2022,ruta)

Archivo: \2022\Costo de Ventas\8. Costo de Ventas Industria CEBE (Agg).xlsx generado con éxito
Archivo: \2022\Costo de Ventas\8. Costo de Ventas Industria CEBE (PLU).xlsx generado con éxito


In [7]:
dfEj.head()

AttributeError: 'NoneType' object has no attribute 'head'